# Scraping Bank1

## 1. Module Imports

In [ ]:
!pip install selenium

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import os
import time
import random
import logging
import csv  
from config import load_config
from log_utils import setup_anonymized_logging
from scraper_utils import add_random_delay
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime, timedelta

In [3]:
# This was taking too long (5 hours for 100 profiles)

# class HumanlikeBehavior:
#     """Helper class for human-like browsing behavior"""
#     @staticmethod
#     def add_human_delay(action_type='page_view'):
#         """Simulate human-like delays based on action type"""
#         delays = {
#             'page_view': (20, 45),  # Time to read a profile
#             'scroll': (2, 5),       # Scrolling delay
#             'click': (1, 3),        # Clicking delay
#             'session_break': (1800, 3600)  # 30-60 minute break between sessions
#         }
#         delay = random.uniform(*delays[action_type])
#         time.sleep(delay)
    
#     @staticmethod
#     def simulate_scrolling(driver):
#         """Simulate human-like scrolling behavior"""
#         total_height = driver.execute_script("return document.body.scrollHeight")
#         viewport_height = driver.execute_script("return window.innerHeight")
#         current_position = 0
        
#         while current_position < total_height:
#             scroll_amount = random.randint(100, 300)
#             current_position = min(current_position + scroll_amount, total_height)
            
#             driver.execute_script(f"window.scrollTo({{top: {current_position}, behavior: 'smooth'}})")
#             HumanlikeBehavior.add_human_delay('scroll')
            
#             if random.random() < 0.1:
#                 scroll_up = random.randint(50, 150)
#                 current_position = max(0, current_position - scroll_up)
#                 driver.execute_script(f"window.scrollTo({{top: {current_position}, behavior: 'smooth'}})")
#                 HumanlikeBehavior.add_human_delay('scroll')



In [2]:
"""
Each profile should take about 10-15 seconds instead of 45-60 seconds
Breaks will be 5-10 minutes instead of 30-60 minutes
It will take breaks every 50 profiles instead of 30
"""

class HumanlikeBehavior:
    """Helper class for human-like browsing behavior"""
    @staticmethod
    def add_human_delay(action_type='page_view'):
        """Simulate human-like delays based on action type"""
        delays = {
            'page_view': (5, 10),    # Reduced from 20-45 to 5-10 seconds
            'scroll': (1, 2),        # Reduced from 2-5 to 1-2 seconds
            'click': (0.5, 1),       # Reduced from 1-3 to 0.5-1 seconds
            'session_break': (300, 600)  # Reduced from 30-60 mins to 5-10 mins
        }
        delay = random.uniform(*delays[action_type])
        time.sleep(delay)
    
    @staticmethod
    def simulate_scrolling(driver):
        """Simulate human-like scrolling behavior"""
        total_height = driver.execute_script("return document.body.scrollHeight")
        viewport_height = driver.execute_script("return window.innerHeight")
        current_position = 0
        
        while current_position < total_height:
            scroll_amount = random.randint(100, 300)
            current_position = min(current_position + scroll_amount, total_height)
            
            driver.execute_script(f"window.scrollTo({{top: {current_position}, behavior: 'smooth'}})")
            HumanlikeBehavior.add_human_delay('scroll')
            
            if random.random() < 0.1:  # Reduced random scrolling up
                scroll_up = random.randint(50, 150)
                current_position = max(0, current_position - scroll_up)
                driver.execute_script(f"window.scrollTo({{top: {current_position}, behavior: 'smooth'}})")
                HumanlikeBehavior.add_human_delay('scroll')

class ScrapingSession:
    def __init__(self):
        self.start_time = datetime.now()
        self.profiles_scraped = 0
        
    def should_take_break(self):
        """Check if we should take a break based on time and profiles scraped"""
        session_duration = datetime.now() - self.start_time
        
        if (session_duration > timedelta(minutes=45) or  # Reduced from 2 hours to 45 minutes
            self.profiles_scraped >= 50):                # Increased from 30 to 50 profiles
            return True
        return False
    
    def take_break(self):
        """Take a break and reset session counters"""
        logger.info("Taking a session break...")
        HumanlikeBehavior.add_human_delay('session_break')
        self.start_time = datetime.now()
        self.profiles_scraped = 0

## 2. Configure logging

In [3]:
# Configure logging
logger = setup_anonymized_logging()

## 3. Loading Configurations and Initialize Session

In [4]:
# Load configuration 
config = load_config()
bank_id = 'bank1'
bank_config = config['banks'][bank_id]

# Initialize session
session = requests.Session()
session.headers.update({
    'User-Agent': random.choice(config['user_agents'])
})

## 4. Utility functions

In [5]:
# Utility functions
def get_total_pages(known_total_donors=478, donors_per_page=15):
    """Calculate total pages needed"""
    return (known_total_donors + donors_per_page - 1) // donors_per_page

def get_scraped_donor_ids():
    """Get list of donor IDs that have already been scraped"""
    try:
        # Try loading from the in-progress file first
        with open('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_bank1_in_progress.json', 'r') as f:
            profiles = json.load(f)
            return {profile['donor_id'] for profile in profiles}
    except FileNotFoundError:
        try:
            # If no in-progress file, try the completed profiles file
            with open('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles.json', 'r') as f:
                profiles = json.load(f)
                return {profile['donor_id'] for profile in profiles}
        except FileNotFoundError:
            return set()

## 5. Donor ID Collection

In [ ]:
## 5. Donor ID Collection

def get_all_donor_ids(session, driver):
    """Get all unique donor IDs from search pages"""
    donor_ids = set()  # Using set to avoid duplicates
    total_pages = 32
    
    try:
        print(f"Starting to collect donor IDs...")
        
        for page in range(1, total_pages + 1):
            url = f"{bank_config['base_url']}/search/?donor_sort=default_Sort&page={page}"
            print(f"Scanning page {page} of {total_pages}")
            
            driver.get(url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "donor-id"))
            )
            
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            donor_id_spans = soup.find_all('span', class_='donor-id')
            
            for span in donor_id_spans:
                donor_id = span.text.strip()
                donor_ids.add(donor_id)
            
            print(f"Found {len(donor_id_spans)} donors on page {page}. Total unique IDs so far: {len(donor_ids)}")
            add_random_delay(5, 10)  # Delay between pages
            
    except Exception as e:
        print(f"Error collecting donor IDs: {str(e)}")
        
    finally:
        print(f"Finished collecting donor IDs. Total unique donors found: {len(donor_ids)}")
        # Print first few IDs as a sample
        sample_ids = list(donor_ids)[:5]
        print(f"Sample of first 5 donor IDs: {sample_ids}")
        return list(donor_ids)

# Initialize Selenium and run collection
print("Initializing Selenium driver...")
options = webdriver.ChromeOptions()
options.add_argument(f'user-agent={random.choice(config["user_agents"])}')
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

try:
    collected_donor_ids = get_all_donor_ids(session, driver)
finally:
    driver.quit()

# Print final results
if collected_donor_ids:
    print(f"\nSuccessfully collected {len(collected_donor_ids)} unique donor IDs")
    # Save IDs to file in bank1 directory
    output_directory = '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1'
    os.makedirs(output_directory, exist_ok=True)
    id_file_path = os.path.join(output_directory, 'donor_ids_bank1.txt') # changed file to donor_ids_bank1.txt for clarity
    with open(id_file_path, 'w') as f:
        for donor_id in collected_donor_ids:
            f.write(f"{donor_id}\n")
    print(f"Saved donor IDs to {id_file_path}")

## 6. Profile Scraping Function

In [6]:

def scrape_profile_bank1(relative_url, driver=None):
    """
    Scrape a single profile page for Bank1 using either Selenium or requests.
    """
    try:
        full_url = f"{bank_config['base_url']}{relative_url}"
        
        if driver:
            # Selenium approach
            driver.get(full_url)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "profile-details"))
            )
            
            # Simulate human reading behavior
            HumanlikeBehavior.simulate_scrolling(driver)
            HumanlikeBehavior.add_human_delay('page_view')
            
            response_text = driver.page_source
            soup = BeautifulSoup(response_text, 'html.parser')
        else:
            # Fallback to requests approach
            response = session.get(full_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
        
        # Initialize profile data dictionary
        profile_data = {
            'url': full_url,
            'donor_id': None,
            'headline': None,
            'donor_description': None,
            'donor_lookalikes': None,
            'height': None,
            'weight': None,
            'eye_color': None,
            'hair_color': None,
            'hair_texture': None,
            'complexion': None,
            'ethnic_origin': None,
            'ancestry': None,
            'religion': None,
            'jewish_ancestry': None,
            'education_level': None,
            'areas_of_study': None
        }
            
        # Extract donor ID
        donor_id_elem = soup.find('span', class_='donor-id')
        if donor_id_elem:
            id_span = donor_id_elem.find('span', class_='id')
        if id_span:
            profile_data['donor_id'] = id_span.text.strip()
        
        # Extract headline
        description_div = soup.find('div', id='description')
        if description_div:
            headline_elem = description_div.find('h2')
        if headline_elem:
            profile_data['headline'] = headline_elem.text.strip()
        
        # Extract donor description
        description_elem = soup.find('div', id='description')
        if description_elem and description_elem.find('p'):
            profile_data['donor_description'] = description_elem.find('p').text.strip()
        
        # Extract look-alikes
        lookalikes_div = soup.find('div', class_='look-a-likes')
        if lookalikes_div:
            profile_data['donor_lookalikes'] = lookalikes_div.text.strip()
            # If the text is empty, check for the "Not available" message
            if not profile_data['donor_lookalikes']:
                not_available = lookalikes_div.find('div', class_='not-available')
                if not_available:
                    profile_data['donor_lookalikes'] = not_available.text.strip()
        
        # Physical characteristics 
        physical_chars = soup.find('section', id='appearance')
        if physical_chars:
            characteristics_div = physical_chars.find('div', id='collapse-panel-1')
            if characteristics_div:
                label_to_key = {
                    'Height:': 'height',
                    'Weight:': 'weight',
                    'Eye Color:': 'eye_color',
                    'Hair Color:': 'hair_color',
                    'Hair Texture:': 'hair_texture',
                    'Complexion:': 'complexion'
                }
                
                for li in characteristics_div.find_all('li'):
                    label_span = li.find('span', class_='tab')
                    if label_span:
                        label = label_span.text.strip()
                        if label in label_to_key:
                            value = li.get_text(strip=True)
                            value = value.replace(label, '').strip()
                            profile_data[label_to_key[label]] = value

        # Heritage information
        heritage_section = soup.find('section', id='heritage')
        if heritage_section:
            heritage_div = heritage_section.find('div', id='collapse-panel-2')
            if heritage_div:
                label_to_key = {
                    'Ethnic Origin:': 'ethnic_origin',
                    'Ancestry (Self-Reported):': 'ancestry',
                    'Religion:': 'religion',
                    'Jewish Ancestry (Self-Reported):': 'jewish_ancestry'
                }
                
                for li in heritage_div.find_all('li'):
                    label_span = li.find('span', class_='tab')
                    if label_span:
                        label = label_span.text.strip()
                        print(f"Found label: '{label}'")  # Debug print
                        if label in label_to_key:
                            # Special IDs for some fields
                            if 'Ethnic Origin:' in label:
                                value_span = li.find('span', id='donor-profile-ethnicity')
                            elif 'Religion:' in label:
                                value_span = li.find('span', id='donor-profile-religiion')
                            elif 'Jewish Ancestry' in label:
                                value_span = li.find('span', id='donor-profile-jewish')
                            else:
                                value_span = li.find('span')
                            
                            if value_span:
                                profile_data[label_to_key[label]] = value_span.text.strip()

        # Education information
        education_section = soup.find('section', id='education')
        if education_section:
            education_div = education_section.find('div', id='collapse-panel-3')
            if education_div:
                label_to_key = {
                    'Education Level:': 'education_level',
                    'Areas of Study:': 'areas_of_study'
                }
                
                for li in education_div.find_all('li'):
                    label_span = li.find('span', class_='tab')
                    if label_span:
                        label = label_span.text.strip()
                        if label in label_to_key:
                            # Get the value directly from the <span> next to the label
                            value_span = li.find_all('span')[1]  # This gets the second <span>, which is the value
                            
                            if value_span:
                                profile_data[label_to_key[label]] = value_span.text.strip()

                    
        return profile_data
            
    except Exception as e:
        logger.error(f"Error scraping profile {full_url}: {str(e)}")
        return None
            

In [7]:
def scrape_donor_profiles(session, page_number, seen_donors, driver=None):
    """Scrape donors from a single page with human-like behavior"""
    should_quit_driver = False
    try:
        # Create driver only if not provided
        if not driver:
            options = webdriver.ChromeOptions()
            options.add_argument(f'user-agent={random.choice(config["user_agents"])}')
            options.add_argument('--headless')  # Optional: run in headless mode
            driver = webdriver.Chrome(options=options)
            should_quit_driver = True
        
        url = f"{bank_config['base_url']}/search/?donor_sort=default_Sort&page={page_number}"
        driver.get(url)
        
        # Wait for content to load and simulate human behavior
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "profile-details"))
        )
        HumanlikeBehavior.simulate_scrolling(driver)
        HumanlikeBehavior.add_human_delay('page_view')
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        donors_on_page = []
        
        profile_sections = soup.find_all('div', class_='profile-details')
        logger.info(f"Found {len(profile_sections)} profiles on page {page_number}")
        
        for profile in profile_sections:
            try:
                donor_id_elem = profile.find('span', class_='donor-id')
                if donor_id_elem:
                    donor_id = donor_id_elem.text.strip()
                    relative_url = f"/donor/{donor_id}"
                    
                    if donor_id not in seen_donors:
                        profile_data = scrape_profile_bank1(relative_url, driver)
                        if profile_data:
                            seen_donors.add(donor_id)
                            donors_on_page.append(profile_data)
                    else:
                        logger.info(f"Found duplicate donor {donor_id} on page {page_number}")
            
            except Exception as e:
                logger.error(f"Error processing profile with ID {donor_id if donor_id else 'unknown'}: {str(e)}")
                continue
        
        if should_quit_driver:
            driver.quit()
        return donors_on_page
        
    except Exception as e:
        logger.error(f"Error scraping page {page_number}: {str(e)}")
        if should_quit_driver and driver:
            driver.quit()
        return []

## 7. Session Management

In [8]:
class ScrapingSession:
    def __init__(self):
        self.start_time = datetime.now()
        self.profiles_scraped = 0
        
    def should_take_break(self):
        """Check if we should take a break based on time and profiles scraped"""
        session_duration = datetime.now() - self.start_time
        
        if (session_duration > timedelta(hours=2) or 
            self.profiles_scraped >= 30):
            return True
        return False
    
    def take_break(self):
        """Take a break and reset session counters"""
        logger.info("Taking a session break...")
        HumanlikeBehavior.add_human_delay('session_break')
        self.start_time = datetime.now()
        self.profiles_scraped = 0

## 8. Test Scraping 

In [ ]:
def test_specific_profiles():
    try:
        # List of known donor IDs to test
        test_donor_ids = ['20035', '19807', '20400']
        profiles = []
        
        # Initialize Selenium for testing
        options = webdriver.ChromeOptions()
        options.add_argument(f'user-agent={random.choice(config["user_agents"])}')
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        
        print("Starting test scrape of specific profiles...")
        
        for donor_id in test_donor_ids:
            relative_url = f"/donor/{donor_id}"
            logger.info(f"Scraping profile for donor {donor_id}")
            profile_data = scrape_profile_bank1(relative_url, driver)
            if profile_data:
                profiles.append(profile_data)
                print(f"Successfully scraped donor {donor_id}")
            else:
                print(f"Failed to scrape donor {donor_id}")
                
        print(f"\nProfiles scraped: {len(profiles)}")
        
        if profiles:
            # Save test results
            test_output_dir = '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw'
            os.makedirs(test_output_dir, exist_ok=True)
            
            with open(os.path.join(test_output_dir, 'test_specific_profiles.json'), 'w', encoding='utf-8') as f:
                json.dump(profiles, f, indent=2, ensure_ascii=False)
            print("Saved test results to raw data folder")
            
        return profiles
            
    except Exception as e:
        logger.error(f"Error in test scraping: {str(e)}")
        print(f"\nError details: {str(e)}")
        return None
    finally:
        if driver:
            driver.quit()

# Run test
test_profiles = test_specific_profiles()

## 9. Scrape and Save

In [9]:
## 9. Scrape and Save

def save_progress(profiles, bank_id):
    """Save intermediate results"""
    output_directory = '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1'
    temp_file = os.path.join(output_directory, f'profiles_{bank_id}_in_progress.json')
    
    # Save as JSON with timestamp in the filename
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_file = os.path.join(output_directory, f'profiles_{bank_id}_backup_{timestamp}.json')
    
    # Save current progress
    with open(temp_file, 'w', encoding='utf-8') as f:
        json.dump(profiles, f, indent=2, ensure_ascii=False)
    
    # Create backup
    with open(backup_file, 'w', encoding='utf-8') as f:
        json.dump(profiles, f, indent=2, ensure_ascii=False)
        
    print(f"Saved progress: {len(profiles)} profiles")
    print(f"Backup created: {backup_file}")

def save_final_results(profiles, bank_id):
    """Save final results in both JSON and CSV formats with versioning"""
    output_directory = '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1'
    os.makedirs(output_directory, exist_ok=True)
    
    if profiles:
        try:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            
            # Save as JSON with timestamp
            json_filename = os.path.join(output_directory, f'profiles_{timestamp}.json')
            with open(json_filename, 'w', encoding='utf-8') as f:
                json.dump(profiles, f, indent=2, ensure_ascii=False)
            
            # Save as CSV with timestamp
            csv_filename = os.path.join(output_directory, f'profiles_{timestamp}.csv')
            with open(csv_filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=profiles[0].keys())
                writer.writeheader()
                writer.writerows(profiles)
            
            # Also save as the main files (overwriting previous versions)
            main_json = os.path.join(output_directory, 'profiles.json')
            main_csv = os.path.join(output_directory, 'profiles.csv')
            
            with open(main_json, 'w', encoding='utf-8') as f:
                json.dump(profiles, f, indent=2, ensure_ascii=False)
            
            with open(main_csv, 'w', newline='', encoding='utf-8') as f:
                writer = csv.DictWriter(f, fieldnames=profiles[0].keys())
                writer.writeheader()
                writer.writerows(profiles)
            
            print(f"Data saved to:")
            print(f"- {json_filename}")
            print(f"- {csv_filename}")
            print(f"- {main_json}")
            print(f"- {main_csv}")
            
        except Exception as e:
            logger.error(f"Error saving final results: {str(e)}")
    else:
        logger.warning("No profiles were collected to save")

def main():
    try:
        # Load configuration
        global config, bank_config
        config = load_config()
        bank_id = 'bank1'
        bank_config = config['banks'][bank_id]

        # Load existing profiles
        try:
            with open('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_bank1_in_progress.json', 'r') as f:
                all_profiles = json.load(f)
            print(f"Loaded {len(all_profiles)} existing profiles from in-progress file")
        except FileNotFoundError:
            all_profiles = []
            print("Starting fresh scrape")

        # Get already scraped donor IDs
        scraped_ids = get_scraped_donor_ids()
        print(f"Found {len(scraped_ids)} already scraped donor IDs")

        # Load all donor IDs
        donor_ids_path = '/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/donor_ids.txt'
        with open(donor_ids_path, 'r') as f:
            all_donor_ids = [line.strip() for line in f]
        
        # Filter out already scraped IDs
        remaining_donor_ids = [id for id in all_donor_ids if id not in scraped_ids]
        print(f"Remaining donors to scrape: {len(remaining_donor_ids)} out of {len(all_donor_ids)}")

        # Initialize session manager
        session_manager = ScrapingSession()

        # Initialize driver
        options = webdriver.ChromeOptions()
        options.add_argument(f'user-agent={random.choice(config["user_agents"])}')
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)

        # Continue scraping remaining profiles
        for i, donor_id in enumerate(remaining_donor_ids, 1):
            try:
                logger.info(f"Scraping profile {i} of {len(remaining_donor_ids)} remaining")
                
                if session_manager.should_take_break():
                    session_manager.take_break()
                
                relative_url = f"/donor/{donor_id}"
                profile_data = scrape_profile_bank1(relative_url, driver)
                
                if profile_data:
                    all_profiles.append(profile_data)
                    session_manager.profiles_scraped += 1
                    
                    # Save progress every 10 profiles
                    if len(all_profiles) % 10 == 0:
                        save_progress(all_profiles, bank_id)
                        print(f"Saved progress: {len(all_profiles)} profiles collected")
                
            except Exception as e:
                logger.error(f"Error scraping profile {donor_id}: {str(e)}")
                # Save progress on error
                save_progress(all_profiles, bank_id)
                # Reinitialize driver on error
                try:
                    driver.quit()
                except:
                    pass
                driver = webdriver.Chrome(options=options)
                continue
            
    except Exception as e:
        logger.error(f"Error during scraping: {str(e)}")
        
    finally:
        try:
            driver.quit()
        except:
            pass
        
        # Save final results
        save_final_results(all_profiles, bank_id)

if __name__ == "__main__":
    main()

Loaded 100 existing profiles from in-progress file
Found 100 already scraped donor IDs
Remaining donors to scrape: 336 out of 436


2024-10-27 12:54:47 - INFO - Scraping profile 1 of 336 remaining
2024-10-27 12:55:37 - INFO - Scraping profile 2 of 336 remaining


Found label: 'Ethnic Origin:'
Found label: 'Ancestry (Self-Reported):'
Found label: 'Religion:'
Found label: 'Jewish Ancestry (Self-Reported):'


2024-10-27 12:56:26 - INFO - Scraping profile 3 of 336 remaining


Found label: 'Ethnic Origin:'
Found label: 'Ancestry (Self-Reported):'
Found label: 'Religion:'
Found label: 'Jewish Ancestry (Self-Reported):'


...

Found label: 'Ethnic Origin:'
Found label: 'Ancestry (Self-Reported):'
Found label: 'Religion:'
Found label: 'Jewish Ancestry (Self-Reported):'


2024-10-27 18:56:20 - INFO - Scraping profile 335 of 336 remaining


Found label: 'Ethnic Origin:'
Found label: 'Ancestry (Self-Reported):'
Found label: 'Religion:'
Found label: 'Jewish Ancestry (Self-Reported):'


2024-10-27 18:57:12 - INFO - Scraping profile 336 of 336 remaining


Found label: 'Ethnic Origin:'
Found label: 'Ancestry (Self-Reported):'
Found label: 'Religion:'
Found label: 'Jewish Ancestry (Self-Reported):'
Found label: 'Ethnic Origin:'
Found label: 'Ancestry (Self-Reported):'
Found label: 'Religion:'
Found label: 'Jewish Ancestry (Self-Reported):'
Data saved to:
- /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_20241027_185803.json
- /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_20241027_185803.csv
- /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles.json
- /Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles.csv


### Cross-checking if backup files match final outputs since scraping was interrupted several times.

In [2]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 20.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 19.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 508.0/508.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 14.9 MB/s eta 0:00:00


In [4]:
import json
import pandas as pd

# Read the JSON files
with open('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_20241027_125054.json', 'r') as f:
    data_1250 = json.load(f)

with open('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_20241027_185803.json', 'r') as f:
    data_1858 = json.load(f)

with open('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles.json', 'r') as f:
    data_main = json.load(f)

# Read the CSV files
df_1250 = pd.read_csv('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_20241027_125054.csv')
df_1858 = pd.read_csv('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles_20241027_185803.csv')
df_main = pd.read_csv('/Users/cindylinsf/Documents/CCI/THESIS/Msc_Thesis_Project_Files/data/raw/bank1/profiles.csv')

# Print the number of profiles in each file
print("Number of profiles in each file:")
print(f"12:50 JSON: {len(data_1250)}")
print(f"12:50 CSV: {len(df_1250)}")
print(f"18:58 JSON: {len(data_1858)}")
print(f"18:58 CSV: {len(df_1858)}")
print(f"Main JSON: {len(data_main)}")
print(f"Main CSV: {len(df_main)}")

# Check if the final files (18:58) match the main files
print("\nDo final files match main files?")
print(f"JSON matches: {len(data_1858) == len(data_main)}")
print(f"CSV matches: {len(df_1858) == len(df_main)}")

# Get unique donor IDs in each version
donor_ids_1250 = set(df_1250['donor_id'])
donor_ids_1858 = set(df_1858['donor_id'])
donor_ids_main = set(df_main['donor_id'])

print("\nNumber of unique donor IDs:")
print(f"12:50 version: {len(donor_ids_1250)}")
print(f"18:58 version: {len(donor_ids_1858)}")
print(f"Main version: {len(donor_ids_main)}")

Number of profiles in each file:
12:50 JSON: 100
12:50 CSV: 100
18:58 JSON: 435
18:58 CSV: 435
Main JSON: 435
Main CSV: 435

Do final files match main files?
JSON matches: True
CSV matches: True

Number of unique donor IDs:
12:50 version: 100
18:58 version: 435
Main version: 435
